# Machine Learning for particle physics discrimination

In this notebook, you will apply your knowledge on machine learning algorithms to discriminate two particle physics processes, mainly e-e+ -> ZHH (1) or ZZH (2), whereas in both cases one Z boson decays to a µ- (antimuon) and µ- (muon). As a check, we first run the analysis on "generator"-level data, i.e. in simulations without any detector response.

For each event, we get the final state particles and assume it is either ZZH or ZHH (properties start with zzh_ or zhh_, respectively). As input parameters, we want to only use the four-vectors of the final state particles (Higgs bosons).

# Data preparation

Download the dataset and import it

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def import_data(path):
    # Load numpy file to pandas dataframe
    df = pd.DataFrame(np.load(path, allow_pickle=True))
    
    # For total ME, sum over helicity final states (for Z boson with spin s=1, 2s+1=3 possibilities, s_z = -1,0,1), average over helicity initial states (RL, LR)
    # Here, however, we just want to summarize the 
    df["zzh_sigmalr"] = 1/2 * ( df["zzh_sigmalrl"] + df["zzh_sigmalrr"] )
    
    return df

In [ ]:
data_mcparticle = ... # Your code here

In [ ]:
data_mcparticle.columns # Take a look at all the columns...

Index(['run', 'event', 'error_code', 'is_zhh', 'is_zzh', 'passed_preselection',
       'true_h1_decay_pdg', 'true_h2_decay_pdg', 'true_z2_decay_pdg',
       'h1_decay_pdg', 'h2_decay_pdg', 'z2_decay_pdg', 'zhh_sigma',
       'zhh_sigmall', 'zhh_sigmalr', 'zhh_sigmarl', 'zhh_sigmarr', 'zhh_mz',
       'zhh_mhh', 'zhh_mzhh', 'zhh_phi', 'zhh_phif', 'zhh_phih',
       'zhh_costheta', 'zhh_costhetaf', 'zhh_costhetah', 'zhh_l1_e',
       'zhh_l1_px', 'zhh_l1_py', 'zhh_l1_pz', 'zhh_l2_e', 'zhh_l2_px',
       'zhh_l2_py', 'zhh_l2_pz', 'zhh_h1_e', 'zhh_h1_px', 'zhh_h1_py',
       'zhh_h1_pz', 'zhh_h2_e', 'zhh_h2_px', 'zhh_h2_py', 'zhh_h2_pz',
       'zzh_sigma', 'zzh_sigmalll', 'zzh_sigmallr', 'zzh_sigmalrl',
       'zzh_sigmalrr', 'zzh_sigmarrr', 'zzh_sigmarrl', 'zzh_sigmarlr',
       'zzh_sigmarll', 'zzh_mz1', 'zzh_mz2', 'zzh_mzz', 'zzh_mzzh', 'zzh_mh',
       'zzh_phi', 'zzh_phiz', 'zzh_phiz1f', 'zzh_phiz2f', 'zzh_costheta',
       'zzh_costhetaz', 'zzh_costhetaz1f', 'zzh_costhetaz2f', 'zzh_

Explanation of four vectors: (e is energy in GeV, pi is impulse for i in (x,y,z) in units of GeV/c^2 )
- xxx_l1_e/pi -> muon
- xxx_l2_e/pi -> antimuon

- zhh_h1_e/pi -> Higgs 1
- zhh_h2_e/pi -> Higgs 2

- zzh_h_e/pi -> Higgs
- zzh_z2f1_e/pi -> particle Z2 decayed to
- zzh_z2f2_e/pi -> anti-particle Z2 decayed to

We want to predict the target column is_zhh, so we get a model, that tells us whether a ZHH or ZZH event occured, given the four-vectors of the assumed final state particles. However, the attached file contains much more data than just the four-vectors (energy + impulse in 3 dimensions) of the final state particles. This can lead to problems, when some data is dependent on some other data in the same event, and oftens leads to a wrongly behaving and generalizing model later. Therefore, using the supplied numpy array data, construct a pandas dataframe with the same data, but...

- delete all properties (columns) including calculated angles (phi, costheta, costheta*) or masses (_m*, _m**, _m***)
- delete all columns including non-numeric data
- delete all columns including calculated matrix elements (they start with zhh_sigma or zzh_sigma, respectively)
- delete all columns including meta-information (run, event, error_code, passed_preselection)
- delete all columns including decay information (true_**_decay_pdg)
- delete all columns including further analysis data (llr, zhh_nll, zzh_nll)
- make sure there is only one target columns that is either 1 or 0

In [ ]:
# Your code here

Also, make sure there are only unique columns and there is no "double" information in an event.

Hint: Take a look at an event (a certain row) and check for duplicates

In [ ]:
# Your code here

# Model training

The aim is to train a boosted-decision-tree (BDT) on a subset of the data to predict the target

In [ ]:
# Split the data in a test and train set; gives X_train, X_test, y_train and y_test
# Check https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

...

In [ ]:
# Certain models might profit from feature scaling, e.g. using StandardScaler and its fit_transform method
# BDTs, however, should in general not require scaling.
# Also check https://stats.stackexchange.com/questions/244507/what-algorithms-need-feature-scaling-beside-from-svm 

In [ ]:
# Create the model instance and fit data and target using the X_train and y_train sets
from sklearn.tree import DecisionTreeClassifier

# Your Code here
# Note: For the start, it will be fine just to use the default parameters of DecisionTreeClassifier

In [ ]:
# Predict the target for the X_test set
y_calc = ...

# Interpretation and Benchmark

To really check, how good the model is and how well it fits our data, one can use different benchmarks. One of the easiest is mean squarred error (MSE).
For a total of n entries of a known (y_known) and calculated (y_calc) property, the MSE is defined as

MSE = (1/n) * sum{from i=1 to n}( [y_calc-y_known]^2 ) 

Also see https://en.wikipedia.org/wiki/Mean_squared_error

In [ ]:
def mse(calc, known):
    ... # Your code here

Calculate the MSE

In [ ]:
mse(y_train, y_calc)

# Outstanding!

# Bonus 1: Confusion Matrix and ROC-curve

These are tools to better visualize where "things go wrong". The confusion matrix shows how many ZZH events are wrongly classified as ZHH and vice-versa and how many are correctly identified. The Receiving-operator-curve shows in a rather intuitive manner how good a certain model can discriminate two cases (here ZZH and ZHH) against each other (diagonal would be equal to 50:50 guessing, i.e. no performance).

Examples of both can be found here:
https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_display_object_visualization.html#sphx-glr-auto-examples-miscellaneous-plot-display-object-visualization-py

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

# Your code here

In [ ]:
from sklearn.metrics import RocCurveDisplay, roc_curve

# Bonus 2: Parameter optimization

In order to fully utilize the power of machine learning algorithms (to get the last remaining percentages of precision/accuracy etc.), it is usually necessary to use the model with the correct hyperparameters (for BDTs, e.g. maximum number of leafs etc). To find those, the easiest way (but rather hard in terms of computing resources) is to do a grid search. At the same time, this process is also crucial to avoid so-called "overfitting", i.e. a model that does not generalize, but instead desperately tries to fit each point, even the outermost outliers.

The procedure is rather long, but well covered in the lecture and tutorials, also here:
https://scikit-learn.org/stable/modules/grid_search.html#grid-search 

from sklearn.model_selection import GridSearchCV

param_grid = {
    "min_samples_split": [2, 5, 10],
    "max_depth": [3, 5, 10]
}
